# Tutorial 2: Forecast after DA


**Goal**: To run an ensemble of forecasts. 
[free_forecast.py ](https://github.com/lkugler/DART-WRF/blob/master/free_forecast.py) contains examples.

Initialize the forecast with either (1) or (2). Run the forecast with (3)
1) Initialize a forecast from defined profiles of temperature, humidity and wind, i.e. from a `wrf_profile` (see WRF guide)
2) Initialize a forecast from an existing forecast, i.e. from WRF restart files - optionally with updates from data assimilation.
3) Run the forecast


### 1) Initialize from sounding profiles

tutorial missing

### 2) Initialize a forecast from a previous forecast
To run a forecast from initial conditions of a previous forecasts, we import these modules
```python
import datetime as dt
from dartwrf.workflows import WorkFlows
```

Let's say you want to run a forecast starting at 9 UTC until 12 UTC.
Initial conditions shall be taken from a previous experiment in `/user/test/data/sim_archive/exp_abc` which was initialized at 6 UTC and there are WRF restart files for 9 UTC.
Then the code would be

```python
prior_path_exp = '/user/test/data/sim_archive/exp_abc'
prior_init_time = dt.datetime(2008,7,30,6)
prior_valid_time = dt.datetime(2008,7,30,9)

w = WorkFlows(exp_config='cfg.py', server_config='srvx1.py')

begin = dt.datetime(2008, 7, 30, 9)
end = dt.datetime(2008, 7, 30, 12)

w.prepare_WRFrundir(begin)

w.prepare_IC_from_prior(prior_path_exp, prior_init_time, prior_valid_time)
```

#### 2b) Optional: Update posterior with increments from assimilation

In order to continue a forecast after assimilation you need the posterior = prior (1) + increments (2)

1. Prepare initial conditions from a prior forecast:
```python
id = w.prepare_IC_from_prior(prior_path_exp, prior_init_time, prior_valid_time, depends_on=id)
```


2. Update the initial conditions from data assimilation:
```python
id = w.update_IC_from_DA(time, depends_on=id)
```

After this, the wrfrst files are updated with assimilation increments from DART output and copied to the WRF's run directories so you can continue to run the forecast ensemble.

### 3) Run the forecast
Define how long you want to run the forecast and when you want WRF-restart files. Since they take a lot of space, we want as few as possible.

```python
timedelta_integrate = dt.timedelta(hours=5)

w.run_ENS(begin=begin,  # start integration from here
          end=time + timedelta_integrate,  # integrate until here
          output_restart_interval=9999,  # do not write WRF restart files
          )
```

If you want to assimilate in 15 minutes again, use
```python
timedelta_integrate = dt.timedelta(hours=5)
timedelta_btw_assim = dt.timedelta(minutes=15)
output_restart_interval = timedelta_btw_assim.total_seconds()/60

id = w.run_ENS(begin=time,  # start integration from here
               end=time + timedelta_integrate,  # integrate until here
               output_restart_interval=output_restart_interval
               )
```
